In [1]:
# That's all the code we need for filling this notebook — after refactoring at least twice...
from intransparent.main import reports_per_country, reports_per_platform, meta_disclosures

reports_per_country(section=1)
disclosures = reports_per_platform(section=2)
meta_disclosures(section=3, disclosures=disclosures)

index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,reports,:,int64,(no nulls)
column,reports_pct,:,float64,(no nulls)


index,iso3,:,category,(no nulls)
index,year,:,category,(no nulls)
column,population,:,int64,(no nulls)
column,population_pct,:,float64,(no nulls)


index,iso3,:,category,(no nulls)
column,country,:,category,(no nulls)
column,region,:,category,(no nulls)


index,,:,int64,(no nulls)
column,superregion,:,category,(no nulls)
column,region,:,category,(no nulls)
column,continent,:,category,(no nulls)


index,,:,int64,(no nulls)
column,iso3,:,category,(no nulls)


...

index,period,:,period[A-DEC],(no nulls)
column,Total,:,Int64,(no nulls)
column,Automattic,:,Int64,(2 nulls)
column,Facebook,:,Int64,(2 nulls)
column,Google/YouTube,:,Int64,(no nulls)
column,Instagram,:,Int64,(2 nulls)
column,LinkedIn,:,Int64,(no nulls)
column,Meta,:,Int64,(2 nulls)
column,Pinterest,:,Int64,(no nulls)
column,Quora,:,Int64,(no nulls)
column,Reddit,:,Int64,(no nulls)


,Meta,Total,Meta Percent
period,,,
2019,"15,884,511","16,836,694",94.344596
2020,"20,307,216","21,447,786",94.682108
2021,"26,885,302","29,157,083",92.208476
2022,"27,190,665","31,802,525",85.498447


,divergent
period,
2020Q4,77
2021Q1,3
2021Q2,4
2022Q2,29


,divergent
period,
2020Q2,67
2020Q3,60
2020Q4,58
2022Q3,18


,divergent
period,
2022Q4,18
